In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Vision_Tech_Data') 

# Actions that we try to detect
actions = np.array(['SIX','WIDE','OUT','FOUR','NO BALL'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'SIX': 0, 'WIDE': 1, 'OUT': 2, 'FOUR': 3, 'NO BALL': 4}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
y = to_categorical(labels).astype(int)

In [13]:
X = np.array(sequences)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [15]:
X.shape

(150, 30, 258)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.callbacks import TensorBoard

In [17]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [18]:

#model = Sequential()
#model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
#model.add(LSTM(128, return_sequences=True, activation='relu'))
#model.add(LSTM(64, return_sequences=False, activation='relu'))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(actions.shape[0], activation='softmax'))

model = Sequential()
model.add(Input(shape=(30, 258)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
model.summary()

Epoch 1/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - categorical_accuracy: 0.2048 - loss: 1.5983
Epoch 2/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - categorical_accuracy: 0.4245 - loss: 1.3465
Epoch 3/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - categorical_accuracy: 0.4497 - loss: 1.1650
Epoch 4/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - categorical_accuracy: 0.5928 - loss: 0.9644
Epoch 5/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - categorical_accuracy: 0.6612 - loss: 0.7787
Epoch 6/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - categorical_accuracy: 0.8821 - loss: 0.5062
Epoch 7/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - categorical_accuracy: 0.5886 - loss: 1.3144
Epoch 8/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - categorical_accuracy: 0.6746 - loss: 0.9902
Epoch 9/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - categorical_accuracy: 0.4987 - loss: 0.9673
Epoch 10/2000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - categorical_accuracy: 0.8461 - loss: 0.7116
Epoch 11/2000
5/5 ━━━━━━━━━━

5/5 [==============================] - 0s 66ms/step - loss: 0.7608 - categorical_accuracy: 0.6549
Epoch 291/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.7503 - categorical_accuracy: 0.6127
Epoch 292/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.6930 - categorical_accuracy: 0.7254
Epoch 293/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.6485 - categorical_accuracy: 0.7254
Epoch 294/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.5832 - categorical_accuracy: 0.7606
Epoch 295/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.5696 - categorical_accuracy: 0.7324
Epoch 296/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.5444 - categorical_accuracy: 0.7958
Epoch 297/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.5071 - categorical_accuracy: 0.8239
Epoch 298/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.4907 - categorical_accuracy: 0.845

5/5 [==============================] - 0s 62ms/step - loss: 0.0861 - categorical_accuracy: 0.9789
Epoch 363/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.0810 - categorical_accuracy: 0.9859
Epoch 364/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.0799 - categorical_accuracy: 0.9859
Epoch 365/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.0789 - categorical_accuracy: 0.9859
Epoch 366/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.0754 - categorical_accuracy: 0.9930
Epoch 367/2000
5/5 [==============================] - 0s 58ms/step - loss: 0.0814 - categorical_accuracy: 0.9789
Epoch 368/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.0700 - categorical_accuracy: 0.9859
Epoch 369/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.0671 - categorical_accuracy: 0.9789
Epoch 370/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.0671 - categorical_accuracy: 0.993

5/5 [==============================] - 0s 62ms/step - loss: 0.6017 - categorical_accuracy: 0.7676
Epoch 435/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.4730 - categorical_accuracy: 0.8521
Epoch 436/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.4584 - categorical_accuracy: 0.8521
Epoch 437/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.4215 - categorical_accuracy: 0.8944
Epoch 438/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.3821 - categorical_accuracy: 0.9225
Epoch 439/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.3458 - categorical_accuracy: 0.8873
Epoch 440/2000
5/5 [==============================] - 0s 74ms/step - loss: 0.3169 - categorical_accuracy: 0.9155
Epoch 441/2000
5/5 [==============================] - 0s 73ms/step - loss: 0.2921 - categorical_accuracy: 0.9085
Epoch 442/2000
5/5 [==============================] - 0s 72ms/step - loss: 0.2784 - categorical_accuracy: 0.922

5/5 [==============================] - 0s 60ms/step - loss: 0.1837 - categorical_accuracy: 0.9437
Epoch 507/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.2450 - categorical_accuracy: 0.9014
Epoch 508/2000
5/5 [==============================] - 0s 60ms/step - loss: 0.1487 - categorical_accuracy: 0.9437
Epoch 509/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.1469 - categorical_accuracy: 0.9507
Epoch 510/2000
5/5 [==============================] - 0s 60ms/step - loss: 0.1213 - categorical_accuracy: 0.9648
Epoch 511/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.1112 - categorical_accuracy: 0.9507
Epoch 512/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.0877 - categorical_accuracy: 0.9718
Epoch 513/2000
5/5 [==============================] - 0s 66ms/step - loss: 0.0840 - categorical_accuracy: 0.9718
Epoch 514/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.0841 - categorical_accuracy: 0.978

...

5/5 [==============================] - 0s 72ms/step - loss: 0.0340 - categorical_accuracy: 0.9859
Epoch 1713/2000
5/5 [==============================] - 0s 72ms/step - loss: 0.0599 - categorical_accuracy: 0.9859
Epoch 1714/2000
5/5 [==============================] - 0s 72ms/step - loss: 0.0302 - categorical_accuracy: 0.9930
Epoch 1715/2000
5/5 [==============================] - 0s 64ms/step - loss: 0.0300 - categorical_accuracy: 0.9930
Epoch 1716/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.0426 - categorical_accuracy: 0.9859
Epoch 1717/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.1083 - categorical_accuracy: 0.9718
Epoch 1718/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.0796 - categorical_accuracy: 0.9648
Epoch 1719/2000
5/5 [==============================] - 0s 64ms/step - loss: 0.0563 - categorical_accuracy: 0.9789
Epoch 1720/2000
5/5 [==============================] - 0s 60ms/step - loss: 0.1856 - categorical_accurac

5/5 [==============================] - 0s 61ms/step - loss: 0.0127 - categorical_accuracy: 1.0000
Epoch 1785/2000
5/5 [==============================] - 0s 60ms/step - loss: 0.0134 - categorical_accuracy: 1.0000
Epoch 1786/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.0167 - categorical_accuracy: 1.0000
Epoch 1787/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.0126 - categorical_accuracy: 1.0000
Epoch 1788/2000
5/5 [==============================] - 0s 60ms/step - loss: 0.0130 - categorical_accuracy: 1.0000
Epoch 1789/2000
5/5 [==============================] - 0s 59ms/step - loss: 0.0130 - categorical_accuracy: 1.0000
Epoch 1790/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.0116 - categorical_accuracy: 1.0000
Epoch 1791/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.0150 - categorical_accuracy: 1.0000
Epoch 1792/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.0110 - categorical_accurac

5/5 [==============================] - 0s 61ms/step - loss: 0.0041 - categorical_accuracy: 1.0000
Epoch 1857/2000
5/5 [==============================] - 0s 64ms/step - loss: 0.0039 - categorical_accuracy: 1.0000
Epoch 1858/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.0041 - categorical_accuracy: 1.0000
Epoch 1859/2000
5/5 [==============================] - 0s 60ms/step - loss: 0.0038 - categorical_accuracy: 1.0000
Epoch 1860/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.0038 - categorical_accuracy: 1.0000
Epoch 1861/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.0038 - categorical_accuracy: 1.0000
Epoch 1862/2000
5/5 [==============================] - 0s 59ms/step - loss: 0.0037 - categorical_accuracy: 1.0000
Epoch 1863/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 1864/2000
5/5 [==============================] - 0s 60ms/step - loss: 0.0036 - categorical_accurac

5/5 [==============================] - 0s 63ms/step - loss: 0.0017 - categorical_accuracy: 1.0000
Epoch 1929/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.0017 - categorical_accuracy: 1.0000
Epoch 1930/2000
5/5 [==============================] - 0s 60ms/step - loss: 0.0017 - categorical_accuracy: 1.0000
Epoch 1931/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.0016 - categorical_accuracy: 1.0000
Epoch 1932/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.0016 - categorical_accuracy: 1.0000
Epoch 1933/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.0015 - categorical_accuracy: 1.0000
Epoch 1934/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.0015 - categorical_accuracy: 1.0000
Epoch 1935/2000
5/5 [==============================] - 0s 65ms/step - loss: 0.0015 - categorical_accuracy: 1.0000
Epoch 1936/2000
5/5 [==============================] - 0s 61ms/step - loss: 0.0014 - categorical_accurac

5/5 [==============================] - 0s 62ms/step - loss: 4.8045e-04 - categorical_accuracy: 1.0000
Epoch 1999/2000
5/5 [==============================] - 0s 62ms/step - loss: 4.7554e-04 - categorical_accuracy: 1.0000
Epoch 2000/2000
5/5 [==============================] - 0s 67ms/step - loss: 4.6748e-04 - categorical_accuracy: 1.0000
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            82688     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[0])]

In [ ]:
actions[np.argmax(y_test[0])]

In [ ]:
model.save('vision.h5')

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # Show to screen
        cv2.imshow('AI CRICKET SCORER', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
number_list = []
ball_list = []
ball = 0
run_sum = 0
wicket_sum = 0
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 10000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 10000)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                            if sentence[-1] == 'SIX':
                                run_sum += 6
                                ball += 1
                            elif sentence[-1] == 'OUT':
                                wicket_sum += 1
                                ball += 1
                            elif sentence[-1] == 'WIDE':
                                run_sum += 1
                                #ball += 0
                            elif sentence[-1] == 'FOUR':
                                run_sum += 4
                                ball += 1
                            elif sentence[-1] == 'NO BALL':
                                run_sum += 0
                                ball += 0
                            txt = " Score: {run:.0f} / {w:.0f}".format(run=run_sum, w=wicket_sum)
                            number_list.append(txt)
                            txt2 = " Overs: {Over:.0f} . {r:.0f}".format(Over=(ball//6), r=(ball%6))
                            ball_list.append(txt2)
                            
                            cv2.waitKey(5000)   
                    else:
                        sentence.append(actions[np.argmax(res)])
                        cv2.waitKey(5000)

            if len(sentence) > 0: 
                sentence = sentence[-1:]
                
            if len(number_list) > 0: 
                number_list = number_list[-1:]
                
            if len(ball_list) > 0: 
                ball_list = ball_list[-1:]


        cv2.rectangle(image, (310,0), (450, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (313,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.rectangle(image, (0,600), (440, 640), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(number_list), (3,630), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.rectangle(image, (0,640), (440, 680), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(ball_list), (3,670), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        #cv2.waitKey(5000)
        
        # Show to screen
        cv2.imshow('AI CRICKET SCORER', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 42ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 49ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 51ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 58ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 48ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 44ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
FOUR
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 41ms/step
OUT
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 47ms/step
OUT
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
SIX
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 53ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 36ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 48ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 53ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 44ms/step
FOUR
<class 'mediapipe.python.solution_

1/1 [==============================] - 0s 44ms/step
FOUR
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 63ms/step
NO BALL
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 45ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 50ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 45ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 52ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 44ms/step
WIDE
<class 'mediapipe.python.sol

1/1 [==============================] - 0s 55ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 46ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 58ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 55ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 47ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 53ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 44ms/step
WIDE
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 63ms/step
WIDE
<class 'mediapipe.python.soluti